# extract k,v pair of institutions from metadata
- calls the ROR REST API for the lat and lng
- uses functools @lru_cache to prevent having to rerun api calls
- created get_ror(x) function to be used on ROR ids. 

In [1]:
import pandas as pd
import functools
from colorama import Fore,Back,Style
import requests
import time
from tqdm import tqdm
from tqdm.gui import tqdm as tqdm_gui
tqdm.pandas(colour='cyan')

In [5]:
# import files
file = "www/LNS_REV_3_limited_metadata.xlsx"

df = pd.read_excel(file, sheet_name='Sheet1')

df2 = df[['id','institution_names']]
df2 = df2.set_index('id') 



In [6]:
# convert the str to list
df2['institution_names2']=df2['institution_names'].apply(lambda x: eval(x))

In [21]:
inst_names_series = df2['institution_names2'].explode()
inst_names_df = inst_names_series.apply(pd.Series)
inst_names_df.columns = ['inst_name','ror_id']
inst_names_df = inst_names_df.drop_duplicates()
print(Fore.LIGHTMAGENTA_EX + f"length of inst_names_df: {len(inst_names_df)}")
inst_names_df = inst_names_df.reset_index()
inst_names_df

length of inst_names_df: 121


,id,inst_name,ror_id
0,https://openalex.org/W2982259685,NaN,NaN
1,https://openalex.org/W2742808058,Nipissing University,https://ror.org/05k14ba46
2,https://openalex.org/W2523280788,Queen's University,https://ror.org/02y72wh86
3,https://openalex.org/W2523280788,Emory University,https://ror.org/03czfpz43
4,https://openalex.org/W2612367365,University of Prince Edward Island,https://ror.org/02xh9x144
...,...,...,...
116,https://openalex.org/W4383710507,Centre for Health Evaluation and Outcome Sciences,https://ror.org/04g6gva85
117,https://openalex.org/W4383710507,Providence Health Care,https://ror.org/03qqdf793
118,https://openalex.org/W4383710507,Vancouver Native Health Society,https://ror.org/04k296d46
119,https://openalex.org/W4383710507,Vancouver Coastal Health Research Institute,https://ror.org/04htzww22


In [9]:

# now query ROR to get lat lon

@functools.lru_cache(maxsize=None) # this is cool - the maxsize number is the number of calls in this case
def get_ror(x:str)->dict:
    #clean ror_id
    if type(x)==str:
        x = x.replace("https://ror.org/","")
        #add to URL
        URL = f"https://api.ror.org/v2/organizations/{x}"
        time.sleep(0.2)
        try:
            response = requests.get(URL, timeout=5)
            response.raise_for_status()
            data = response.json()
            if data:
                bingo = data.get('locations','nah')
                if bingo:
                    lat = bingo[0].get('geonames_details','no geo').get('lat','no lat')
                    lng = bingo[0].get('geonames_details','no geo').get('lng','no lng')
                    return pd.Series({'lat':lat,'lng':lng})
                else:
                    return pd.Series({'lat':None,'lng':None})
            else:
                return pd.Series({'lat':None,'lng':None})

        except (requests.exceptions.HTTPError,requests.exceptions.ConnectionError,requests.exceptions.Timeout,requests.exceptions.RequestException) as err:
            print(f"An error occurred: {err}")
            return pd.Series({'lat':None,'lng':None})
    else:
        return pd.Series({'lat':None,'lng':None})



In [12]:
a = inst_names_df.iloc[6,2]
print(a)
get_ror(a)

https://ror.org/0160cpw27


lat     53.55014
lng   -113.46871
dtype: float64

In [15]:
# apply to new df
df_ror = inst_names_df['ror_id'].progress_apply(get_ror)
df_ror

100%|██████████| 121/121 [00:00<00:00, 25722.80it/s]


,lat,lng
0,NaN,NaN
1,46.31680,-79.46633
2,44.22976,-76.48098
3,33.74900,-84.38798
4,46.23459,-63.12560
...,...,...
116,49.24966,-123.11934
117,49.24966,-123.11934
118,49.24966,-123.11934
119,49.24966,-123.11934


In [22]:
# merge inst_names_df with df_ror

inst_names_df = inst_names_df.merge(df_ror, left_index=True,right_index=True,suffixes=("_x","_y"))
inst_names_df

,id,inst_name,ror_id,lat,lng
0,https://openalex.org/W2982259685,NaN,NaN,NaN,NaN
1,https://openalex.org/W2742808058,Nipissing University,https://ror.org/05k14ba46,46.31680,-79.46633
2,https://openalex.org/W2523280788,Queen's University,https://ror.org/02y72wh86,44.22976,-76.48098
3,https://openalex.org/W2523280788,Emory University,https://ror.org/03czfpz43,33.74900,-84.38798
4,https://openalex.org/W2612367365,University of Prince Edward Island,https://ror.org/02xh9x144,46.23459,-63.12560
...,...,...,...,...,...
116,https://openalex.org/W4383710507,Centre for Health Evaluation and Outcome Sciences,https://ror.org/04g6gva85,49.24966,-123.11934
117,https://openalex.org/W4383710507,Providence Health Care,https://ror.org/03qqdf793,49.24966,-123.11934
118,https://openalex.org/W4383710507,Vancouver Native Health Society,https://ror.org/04k296d46,49.24966,-123.11934
119,https://openalex.org/W4383710507,Vancouver Coastal Health Research Institute,https://ror.org/04htzww22,49.24966,-123.11934


In [ ]:
# need to replace NaN with None
inst_names_df = inst

In [23]:


# save out
inst_names_df.to_csv("www/inst_names.csv", encoding="utf-8")


In [ ]:
#get_ror.cache_clear()  # This will clear the cache